# Simulation of Belvarafenib and Cobimetinib inhibition in BRAF<sup>V600E</sup> melanoma cells using the MARM2 model

Here you can simulate the respose of BRAF<sup>V600E</sup> A375 melanoma cells under different doses of RAF (belvarafenib) and MEK inhibitors (cobimetinib). 

**Note**: this code performs the simulation for steady state responses under different inhibitor conditions. Use the Jupyter Notebook *Simulate_Belva_Cobi_Traj_MARM2_BRAF_V600E.ipynb* to generate simulation results for time course trajectories. 

## Import of libraries
Importing libraries necessary to run MARM2 model simulations.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy as cp
import itertools
import os
from pathlib import Path

Changes to main file directory. This works provided the "main_dir" has the correct directory name and the cwd starts within the main file directory. This might require tweaking under some high performance computing setups.

In [2]:
main_dir = "MARM2_Andrew_8_24"
for i in Path().resolve().parents:
    if i.parts[-1] == main_dir:
        os.chdir(i)

Importing the MARM2 PySB model and the simulator.  

In [3]:
from pysb.simulator import ScipyOdeSimulator
from pysb.core import as_complex_pattern
from pysb.bng import generate_equations

from scripts.models.MARM2_BRAF_V600E_No_RTK import model

## User-defined experimental setup
In this section you can alter the setup of the experiment simulated by MARM2. First, you need to define the experimental setup of the pre-treatment phase and of the subsequent inhibitor dose phase. The variables needed for the pre-treatment phase are:

<b>Pretreatment_time (h)</b>: defines the duration of the pre-treatment phase.
    
For the treatment phase, you need to set the running time of simulation after inhibitor dose. This is done with the following variables:

<b>Simulation_time (h)</b>: define the simulation time after inhibitor dose. 

<b>N_time_points</b>: define the number of time points returned by each model simulation. 

1. **Pretreatment duration** in hours.

In [4]:
t_pretrt = 24

2. **Simulation time** in hours.

In [5]:
t_trt = 24

3. **Parameter set** selects which of the 50 best-fit parameter sets to use for the simulation. Set 0 is the best fit and 49 the worst.

In [6]:
param_set_index = 13

4. **N_time_points** defineds the number of time points returned by each individual model simulation

In [7]:
N_time_points = 97

## Generate model equations
PySB runs BioNetGen to generate the reaction network

In [8]:
generate_equations(model)

## Parameter set preparation

Loads and prepares the parameter sets originally generated for A375 cell line described in Fr&ouml;hlich et al https://www.embopress.org/doi/full/10.15252/msb.202210988. While the RAF inhibitor for that paper was Vemurafenib the parameters of the inhibitor are altered to align it with Belvarafenib.

In [9]:
param_sets = pd.read_csv(Path('data/parameter_data/RTKERK_pRAF_EGF_EGFR_MEKi_PRAFi_RAFi.csv'), index_col=0)
# finds the parameters of the .csv file that correspond to Cobimetinib and Vemurafenib and maps them to MEKi and RAFi (respectively)
rename_dict = {}
for i in param_sets.columns:
    if "Cobimetinib" in i or "Vemurafenib" in i:
        rename_dict[i] = i.replace("Cobimetinib","MEKi").replace("Vemurafenib","RAFi")
param_sets = param_sets.rename(columns = rename_dict)

# finds the parameters which are stored in .csv file but not in the model and removes them
csv_spec_params = set(param_sets.columns)-(set(param_sets.columns)&set([i.name for i in model.parameters]))
param_sets = param_sets.drop(csv_spec_params, axis=1)

# Removes preference against second inhibitor binding to model pan RAF inhibitor
param_sets["ep_RAF_RAF_mod_RAFi_double_ddG"] = 0

params = param_sets.iloc[param_set_index].to_dict()

params['RAFi_0'] = 0.0
params['MEKi_0'] = 0.0

## Simulations

First we define some utility functions that will be used below.
*Equilibrate* runs a model simulation till steady state for that parameter set.
*Get_species_index* find and retunrs the index of speies in the model given input specie patterns. 

In [10]:
def equilibrate(simulator, initials,verbose = True):
    """Simulate a model from given initial conditions until it reaches steady state"""
    scale = 10
    t_start = 10
    df = None
    tspan = np.geomspace(t_start, t_start * scale)
    while True:
        if verbose:
            print(f"    at t={tspan[-1]:<5.3g} ... ", end='', flush=True)
        res = simulator.run(tspan=tspan, initials=initials)
        df = pd.concat([df, res.dataframe.iloc[1:]])
        initials = res.species[-1]
        close = np.isclose(
            *res.species[[-1,-2]].view(float).reshape(2,-1),
            rtol=1e-3
        )
        cs = np.sum(close)
        n = len(simulator.model.species)
        if verbose:
            print(f"{cs}/{n} species converged")
        if np.all(close):
            break
        tspan *= scale
    return df

In [11]:
def get_species_index(model, pattern):
    """Return the integer species number for a given species in the model"""
    pattern = as_complex_pattern(pattern)
    matches = [
        i for i, s in enumerate(model.species)
        if s.is_equivalent_to(pattern)
    ]
    n = len(matches)
    assert n == 1, f"Expected exactly one match, got {n}"
    return matches[0]

## Initial equilibrium
First we run the model from its baseline initial conditions until equilibrium is reached. For example protein synthesis/degradation, phosphorylation/dephosphorylation, and drug binding/unbinding all need to reach steady state to match the state of the cells in the experimental setup. There may be some time without visible progress as behind the scenes PySB runs BioNetGen to generate the reaction network and Cython to compile the resulting differential equations into efficient executable code.

In [12]:
sim = ScipyOdeSimulator(model,param_values=params) 
df_eq = equilibrate(sim, None)

    at t=100   ... 875/875 species converged


Now that the model has been simulated once and the actual molecular species have been enumerated, we can find the exact species numbers for the inhibitors. These are needed so that their concentrations can be overridden in the model state for subsequent simulations.

In [13]:
RAFi_index = get_species_index(model, model.monomers.RAFi(raf=None)**model.compartments.CP)
MEKi_index = get_species_index(model, model.monomers.MEKi(mek=None)**model.compartments.CP)

## Inhibitor pre-treatment

We take the final state of the equilibration simulation and use it as the initial state of this new simulation, overriding the RAFi and MEKi concentrations with pre-specified values.

In [14]:
initials_pre = df_eq.iloc[-1, :len(model.species)].copy()
initials_pre[RAFi_index] = 0.0
initials_pre[MEKi_index] = 0.0

#fixed time pre-treatment simulation
tspan_pretrt = np.linspace(0, t_pretrt, N_time_points)
df_pre=sim.run(tspan=tspan_pretrt, initials=initials_pre.to_list()).dataframe

#run pre-tretment to steady state instead of using specified time  
#df_pre = equilibrate(sim, initials_pre)

In case the previous simulation was run to steady state, we want to retain only the first t_pretrt hours of pre-treatment plus the state at final equilibrium. So we cut the time series down using a Pandas slice operation and adjust the remaining time values to begin at -pre_time_max.

In [15]:
if (len(df_pre.loc[:t_pretrt])<len(df_pre)):
   df_pre_tmp = df_pre.loc[:t_pretrt]
   df_pre_tmp.iloc[-1] = df_pre.iloc[-1]
   df_pre= df_pre_tmp
df_pre['time'] = df_pre.index
df_pre['time'] = df_pre['time']-t_pretrt
df_pre['time'].iloc[-1] = 0
df_pre.reset_index(drop=True, inplace=True)
df_pre.set_index('time', inplace=True)

## Inhibitor treatment

We run another simulation starting from the final state of the pre-treatment simulation, overriding the MEKi and 
RAFi concentrations with a range of inhibitor values. Each condition is run to equilibrium.

In [16]:
#set the dilution range for the RAF inhibitor, which is x axis
RAFi_dil=np.logspace(-2.25,.5, 9); #uM
RAFi_dil = np.concatenate(([0],RAFi_dil))
#set the dilution range for the MEK inhibitor, which is y axis
MEKi_dil=np.logspace(-2.75,0, 9); #uM
MEKi_dil=np.concatenate(([0],MEKi_dil))

# Moved from using f and g to single and double ddG values (respectively), currently reads from model, but can be perturbed if needed
s_ddG=[params["ep_RAF_RAF_mod_RAFi_single_ddG"][0]];
d_ddG=[params["ep_RAF_RAF_mod_RAFi_double_ddG"][0]];
fgtitle=['pan_RAF'];

In [17]:
def simulate_inhib_dose(dose_info):
    params.update({'RAFi_0': dose_info[0],'MEKi_0': dose_info[1]});
    #run this to assure model is run to steady_ state, 
    res = equilibrate(ScipyOdeSimulator(model,param_values=cp.deepcopy(params)) , None,verbose=False)
    print(1,end="")
    return [dose_info, res.iloc[-1]]

In [18]:
#define observables to plot
plt_obs=['pMEK', 'pMEK_obs', 'pERK', 'pERK_obs'];
dr_df = pd.DataFrame(columns = ["RAFi_0_uM","MEKi_0_uM"]+plt_obs)
dose_ind = list(itertools.product(*[RAFi_dil,MEKi_dil]))
dr_df["RAFi_0_uM"] = [i[0] for i in dose_ind]
dr_df["MEKi_0_uM"] = [i[1] for i in dose_ind]
dr_df = dr_df.set_index(["RAFi_0_uM","MEKi_0_uM"])
results = [simulate_inhib_dose(i) for i in dose_ind]
for i in range(len(results)):
    dr_df.loc[results[i][0]] = results[i][1][plt_obs]
dr_df.to_csv(Path("data/dose_response_dtf/Belva_Cobi_SS_BRAF_V600E"))

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [20]:
!conda env export --name quant_bio

name: quant_bio
channels:
  - conda-forge
  - alubbock
  - defaults
dependencies:
  - anyio=4.2.0=py311hca03da5_0
  - appnope=0.1.2=py311hca03da5_1001
  - argon2-cffi=21.3.0=pyhd3eb1b0_0
  - argon2-cffi-bindings=21.2.0=py311h80987f9_0
  - asttokens=2.0.5=pyhd3eb1b0_0
  - async-lru=2.0.4=py311hca03da5_0
  - attrs=23.1.0=py311hca03da5_0
  - babel=2.11.0=py311hca03da5_0
  - beautifulsoup4=4.12.2=py311hca03da5_0
  - bionetgen=2.9.1=0
  - blas=1.0=openblas
  - bleach=4.1.0=pyhd3eb1b0_0
  - bottleneck=1.3.7=py311hb9f6ed7_0
  - brotli=1.0.9=h1a28f6b_7
  - brotli-bin=1.0.9=h1a28f6b_7
  - brotli-python=1.0.9=py311h313beb8_7
  - bzip2=1.0.8=h80987f9_5
  - ca-certificates=2024.2.2=hf0a4a13_0
  - certifi=2024.2.2=py311hca03da5_0
  - cffi=1.16.0=py311h80987f9_0
  - charset-normalizer=2.0.4=pyhd3eb1b0_0
  - comm=0.2.1=py311hca03da5_0
  - contourpy=1.2.0=py311h48ca7d4_0
  - cycler=0.11.0=pyhd3eb1b0_0
  - cython=3.0.8=py311h80987f9_0
  - debugpy=1.6.7=py311h313beb8_0
  - decorator=5.1.1=pyhd3eb1b0_0
 